# `Материалы кафедры ММП факультета ВМК МГУ. Введение в Эффективные Системы Глубокого Обучение.`

# `Семинар 06.3. Граф вычислений с точки зрения эффективности: выполнение CUDA операций`

### `Материалы составил Феоктистов Дмитрий (@trandelik)`

#### `Москва, Осенний семестр 2025`

О чём можно узнать из этого ноутбука:

* `Cuda Events` и их использование для замера времени
* `Cuda Streams` для достижения overlap
* `Cuda Graphs` для уменьшения overhead-а API

### `Setup`

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import torch
import torch.nn as nn
import time
import os

from torch.profiler import profile, record_function, ProfilerActivity

if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. This notebook requires a GPU.")

device = torch.device("cuda")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.layers(x)

In [4]:
log_dir = "profiler_logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    print(f"Created directory: {log_dir}")

### `CUDA Events`

`CUDA Events` — это маркеры в потоке выполнения CUDA, которые позволяют **точно измерять время выполнения операций на GPU**. Операции CUDA по своей природе **асинхронны**. Когда вы вызываете `model(data)`, Python немедленно возвращает управление вашему скрипту, в то время как фактические вычисления ставятся в очередь для выполнения на GPU.

Использование `time.time()` для измерения производительности GPU некорректно, так как оно измеряет только время, которое CPU тратит на *запуск* операции, а не время, которое GPU тратит на её *выполнение*.

Правильным инструментом является `torch.cuda.Event`. Алгоритм работы:
1. Создать начальное и конечное события: `start = torch.cuda.Event(enable_timing=True)`.
2. Записать начальное событие: `start.record()`.
3. Выполнить операции на GPU.
4. Записать конечное событие: `end.record()`.
5. Синхронизировать CPU, чтобы дождаться завершения событий: `torch.cuda.synchronize()`.
6. Получить время: `elapsed_ms = start.elapsed_time(end)`.

In [5]:
input_size = 10240
hidden_size = 1024
output_size = 512
batch_size = 1280
num_batches = 1

model = SimpleModel(input_size, hidden_size, output_size).to(device)
cpu_data = [torch.randn(batch_size, input_size, pin_memory=True) for _ in range(num_batches)]

dummy_input = torch.randn(batch_size, input_size, device=device)

In [6]:
print("Warming up GPU...")
for _ in range(10):
    _ = model(dummy_input)
torch.cuda.synchronize()

# --- 1. The WRONG way (using time.time) ---
# This measures CPU launch time, which is very small and misleading
t0 = time.time()
_ = model(dummy_input)
t1 = time.time()
print(f"Incorrect time (time.time() without sync): {(t1 - t0) * 1000:.4f} ms")
torch.cuda.synchronize()

# --- 2. The CORRECT way (using CUDA Events) ---
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)
start_event.record()
_ = model(dummy_input)
end_event.record()
torch.cuda.synchronize()

elapsed_time_ms = start_event.elapsed_time(end_event)
print(f"Correct time (torch.cuda.Event): {elapsed_time_ms:.4f} ms")

Warming up GPU...
Incorrect time (time.time() without sync): 5.2507 ms
Correct time (torch.cuda.Event): 2.4279 ms


### `Baseline`

В оставшейся части этого ноутбука мы будем использовать `time.time()` для замера времени всего цикла инференса по батчам с вызовом `torch.cuda.synchronize()` в конце, чтобы сделать замер более честным. Так измеряется полное реальное время выполнения, что является корректным способом сравнения общей производительности различных стратегий.

In [7]:
def synchronous_execution(cpu_data, model, num_batches):
    for data in cpu_data:
        data_gpu = data.to(device)
        output_gpu = model(data_gpu)
    torch.cuda.synchronize()

### `CUDA Streams`

`CUDA Stream` (поток CUDA) — это последовательность операций, выполняемых на GPU в определенном порядке. По умолчанию в PyTorch используется один глобальный поток (`default stream`). Используя несколько потоков, мы можем попросить GPU выполнять независимые последовательности операций **одновременно**. 

Основная цель — **совместить** передачу данных и вычисления, чтобы скрыть задержки и максимально загрузить GPU, который в противном случае простаивал бы в ожидании данных.

Ключевые элементы:
- **`torch.cuda.Stream()`**: Создает новый, не-дефолтный поток.
- **`with torch.cuda.stream(s):`**: Контекстный менеджер, который направляет все CUDA-операции внутри блока в указанный поток `s`.
- **`non_blocking=True`**: Важнейший аргумент для метода `.to()`, который делает копирование данных асинхронным.

In [8]:
def streamed_execution(cpu_data, model, num_batches):
    compute_stream = torch.cuda.Stream()
    data_stream = torch.cuda.Stream()
        
    for i in range(num_batches):
        with torch.cuda.stream(data_stream):
            data_gpu = cpu_data[i].to(device, non_blocking=True)
        with torch.cuda.stream(compute_stream):
            output_gpu = model(data_gpu)
    torch.cuda.synchronize()

In [9]:
def streamed_execution_v2(cpu_data, model, num_batches):
    compute_stream = torch.cuda.Stream()
    data_stream = torch.cuda.Stream()
    with torch.cuda.stream(data_stream):
        data_gpu = cpu_data[0].to(device, non_blocking=True)
        
    for i in range(num_batches):
        compute_stream.wait_stream(data_stream)
        with torch.cuda.stream(compute_stream):
            output_gpu = model(data_gpu)
        if i < num_batches - 1:
            with torch.cuda.stream(data_stream):
                data_gpu = cpu_data[i + 1].to(device, non_blocking=True)
    torch.cuda.synchronize()

### `CUDA Graphs`

Даже при использовании потоков CPU все равно должен отправлять на запуск каждое отдельное ядро (kernel) для каждого батча. Этот процесс запуска имеет небольшие, но ненулевые накладные расходы. Для моделей с множеством мелких операций или на очень быстрых GPU эти накладные расходы CPU могут стать узким местом.

**CUDA Graphs** решают эту проблему, позволяя «захватить» последовательность операций GPU и затем «перезапускать» её одной командой. Драйвер GPU может оптимизировать этот предопределенный граф для чрезвычайно быстрого выполнения, почти полностью устраняя накладные расходы CPU на запуск ядер.

**Ключевые ограничения:**
- Операции внутри графа (архитектура модели) должны быть статичными.
- **Размеры** входных и выходных тензоров не должны меняться между воспроизведениями.

План работы:
1.  **Warmup**: запустить код обычным способом пару рах.
2.  **Capture**: создать статичные тензоры для входа/выхода. Записать запуск модели в `torch.cuda.CUDAGraph`.
3.  **Replay**: В цикле копировать новые данные в статичный вход и звать `graph.replay()`.

In [10]:
def prepare_graph(batch_size, input_size, model):
    # 1. Define static tensors. Their memory locations will be captured by the graph.
    static_input = torch.randn(batch_size, input_size, device=device)
    
    # 2. Warmup: Run the model once to prepare the GPU and caches.
    # This is important before capturing the graph.
    torch.cuda.synchronize()
    for _ in range(3):
        _ = model(static_input)
    torch.cuda.synchronize()

    # 3. Capture the graph
    g = torch.cuda.CUDAGraph()
    with torch.cuda.graph(g):
        static_output = model(static_input)
    torch.cuda.synchronize()
    return g, static_input

def run_graph(cpu_data, graph, static_input):
    torch.cuda.synchronize()
    for data in cpu_data:
        # Copy new data into the static input tensor
        static_input.copy_(data) 
        # Replay the captured graph. This is much faster than launching kernels individually.
        graph.replay()
    torch.cuda.synchronize()

#### `В чем разница между CUDA Graphs и torch.compile?`

Хотя обе технологии нацелены на ускорение, они работают на разных уровнях и решают немного разные задачи:

| Характеристика | `torch.cuda.graph` (ручное использование) | `torch.compile()` |
| :--- | :--- | :--- |
| **Основная цель** | Устранение накладных расходов CPU на запуск ядер. | Комплексная JIT-компиляция модели: слияние операций (fusion), генерация эффективных ядер, минимизация Python overhead. |
| **Уровень** | Низкоуровневый API CUDA. | Высокоуровневый API PyTorch. |
| **Гибкость** | Очень низкая. Требует статической архитектуры и **статических размеров тензоров**. | Высокая. Может обрабатывать динамические размеры тензоров (с перекомпиляцией) и условную логику. |
| **Простота** | Требует ручной настройки, разогрева, захвата и воспроизведения. | Просто обернуть модель: `compiled_model = torch.compile(model)`. |
| **Когда использовать** | В inference-циклах, где задержка запуска CPU является доказанным узким местом. | Почти всегда. Это рекомендуемый способ ускорения моделей в PyTorch 2.x. `torch.compile` **может использовать CUDA Graphs внутри себя** как одну из стратегий оптимизации. |

`torch.compile` — это универсальный и предпочтительный инструмент. Ручное использование `CUDA Graphs` — это более специализированная техника для достижения максимальной производительности в очень специфических, статичных сценариях.

### `Сравнение производительности`

In [11]:
def compare_performance(batch_size, input_size=1024, hidden_size=1024, output_size=512, num_batches=1280):
    print(f"\n--- Сравнение производительности для batch_size = {batch_size} ---")

    model = SimpleModel(input_size, hidden_size, output_size).to(device)
    # pin_memory - важно для асинхронной передачи данных
    cpu_data = [torch.randn(batch_size, input_size, pin_memory=True) for _ in range(num_batches)]

    print("Подготовка и разогрев GPU...")
    for _ in range(10):
        _ = model(torch.randn(batch_size, input_size, device=device))
    torch.cuda.synchronize()

    print("Измерение синхронного выполнения (Baseline)...")
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof_sync:
        with record_function(f"synchronous_run_{batch_size}"):
            start_time = time.time()
            synchronous_execution(cpu_data=cpu_data, model=model, num_batches=num_batches)
            sync_duration = time.time() - start_time
    prof_sync.export_chrome_trace(os.path.join(log_dir, f"sync_trace_{batch_size}.json"))
    
    print("Измерение выполнения с CUDA Streams...")
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof_async:
        with record_function(f"streamed_run_{batch_size}"):
            start_time = time.time()
            streamed_execution(cpu_data=cpu_data, model=model, num_batches=num_batches)
            stream_duration = time.time() - start_time
    prof_async.export_chrome_trace(os.path.join(log_dir, f"stream_trace_{batch_size}.json"))


    print("Измерение выполнения с CUDA Graphs...")
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof_graph:
        with record_function("cuda_graph_run"):
            graph, static_input = prepare_graph(batch_size=batch_size, input_size=input_size, model=model)
            start_time = time.time()
            run_graph(cpu_data=cpu_data, graph=graph, static_input=static_input)
            graph_duration = time.time() - start_time
    
    prof_graph.export_chrome_trace(os.path.join(log_dir, f"graph_trace_{batch_size}.json"))

    print("\n--- Итоги ---")
    print(f"Синхронное (Baseline):  {sync_duration:.4f} секунд")
    print(f"CUDA Streams:           {stream_duration:.4f} секунд (Ускорение: {sync_duration / stream_duration:.2f}x)")
    print(f"CUDA Graphs:            {graph_duration:.4f} секунд (Ускорение: {sync_duration / graph_duration:.2f}x)")

In [12]:
compare_performance(batch_size=1280)


--- Сравнение производительности для batch_size = 1280 ---
Подготовка и разогрев GPU...
Измерение синхронного выполнения (Baseline)...
Измерение выполнения с CUDA Streams...
Измерение выполнения с CUDA Graphs...

--- Итоги ---
Синхронное (Baseline):  1.2326 секунд
CUDA Streams:           0.8318 секунд (Ускорение: 1.48x)
CUDA Graphs:            1.1350 секунд (Ускорение: 1.09x)


In [13]:
compare_performance(batch_size=16)


--- Сравнение производительности для batch_size = 16 ---
Подготовка и разогрев GPU...
Измерение синхронного выполнения (Baseline)...
Измерение выполнения с CUDA Streams...
Измерение выполнения с CUDA Graphs...

--- Итоги ---
Синхронное (Baseline):  0.5552 секунд
CUDA Streams:           0.6231 секунд (Ускорение: 0.89x)
CUDA Graphs:            0.1401 секунд (Ускорение: 3.96x)


In [14]:
compare_performance(batch_size=1)


--- Сравнение производительности для batch_size = 1 ---
Подготовка и разогрев GPU...
Измерение синхронного выполнения (Baseline)...
Измерение выполнения с CUDA Streams...
Измерение выполнения с CUDA Graphs...

--- Итоги ---
Синхронное (Baseline):  0.4950 секунд
CUDA Streams:           0.5571 секунд (Ускорение: 0.89x)
CUDA Graphs:            0.1029 секунд (Ускорение: 4.81x)
